# Phase 1, Flase Flag Data

## Training the model

In [120]:
import numpy as np
import pandas
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor,BaggingRegressor
URL = 'C:\Users\Kowshik\Desktop\Lighthouse\Consolidated data\MTV_T.xlsx'
df_train = pandas.read_excel(URL,sheetname=1)
df_total = pandas.read_excel(URL,sheetname=0)
df_test = pandas.read_excel(URL,sheetname=2)

In [121]:
df_train_false = df_train[:]
df_train_false = df_train_false[df_train_false.Level != True]
#df_train_false = df_train_false[np.isfinite(df_train_false['Impressions'])]
#deleting unwanted columns
del df_train_false['BreakCode']                                  
del df_train_false['EvType']
del df_train_false['PromoCategory']
del df_train_false['PromoType']
del df_train_false['EndTime']
del df_train_false['Level']
#######################################################################################Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_train_false['Description'].values)-1):
    priorProgram.append(df_train_false['Description'].values[i-1])
    subsequentProgram.append(df_train_false['Description'].values[i+1])
df_train_false =df_train_false.drop(df_train_false.index[[0]])
df_train_false =df_train_false.drop(df_train_false.index[[len(df_train_false['Description'].values)-1]])
df_train_false['priorProgram'] = priorProgram 
df_train_false['subsequentProgram'] =  subsequentProgram
########################################################################################Categerizing the channels
df_PR = df_total[:]
df_PR = df_PR[df_PR.Level != True]
df_PR = df_PR[['Description','Impressions','ProgrammeTheme']]

A = np.unique(df_PR['Description'].values)
PI = {}
for i in A:
    PI[i] = []
    for j in range(len(df_PR['Description'].values)):
        if df_PR['Description'].values[j] == i:
            PI[i].append(df_PR['Impressions'].values[j])
PI_AVG = {}
for i in A:
    PI_AVG[i] = np.mean(PI[i])
    
clus = []
for i in PI_AVG.values()[1:]:
    clus.append([i])
from sklearn.cluster import KMeans
clu = KMeans(n_clusters=6,n_init = 100,precompute_distances=False,n_jobs=1) #----------------- NO OF CLUSTERS ------------------>
clu.fit(clus)
Description_classified_prior = df_train_false['priorProgram'].values
for i in range(len(df_train_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_train_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_train_false['subsequentProgram'].values
for i in range(len(df_train_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_train_false['subsequentProgram'].values[i]])[0]
df_train_false['priorProgram'] = Description_classified_prior
df_train_false['subsequentProgram'] = Description_classifiedsubsequent
df_train_false.priorProgram= df_train_false.priorProgram.astype(str) 
df_train_false.subsequentProgram= df_train_false.subsequentProgram.astype(str)



##################################################################################################Deleting and string conversion
df_train_false.Date = df_train_false.Date.astype(str)                    #Changing the Date as String 
df_train_false.ProgrammeTheme= df_train_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_train_false.ProgrammeGenre= df_train_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_train_false.StartTime= df_train_false.StartTime.astype(str) 
df_train_false.Length= df_train_false.Length.astype(str)
Y_train_1 = np.array(df_train_false.Impressions)                       #Creating a array for impression Prediction 
del df_train_false['Description']




#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_false['Length'].values]
df_train_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_train_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_train_false['Date'].values]
df_train_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_train_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_train_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_train_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_train_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_train_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_train_false['StartTime'].values
for i in range(len(df_train_false['StartTime'].values)):
    if len(df_train_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_false['StartTime'].values[i].split()[1]
df_train_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_train_false['hour'] = [map(int,i.split(':'))[0] for i in df_train_false['StartTime'].values]#--------------------hour------------------------------->
df_train_false.hour= df_train_false.hour.astype(str)
df_train_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_train_false['StartTime'].values]


Now Train Data is ready for training the model. encoding should be done with both testing and training data.

## Testing the phase 1 model

In [122]:
df_test_false = df_test[:]
#df_test_false.Level= df_test_false.Level.astype(str)
#Dropping true Flag values
df_test_false = df_test_false[df_test_false.Level != True]
df_test_false['D'] = df_test_false['Date'].values################################################ # # # # # # # # # #  # # # #
#deleting unwanted columns
del df_test_false['BreakCode']                                  
del df_test_false['EvType']
del df_test_false['PromoCategory']
del df_test_false['PromoType']
del df_test_false['EndTime']
del df_test_false['Level'] 
#Taggin 'Movies / Plays' into Movie

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_test_false['Description'].values)-1):
    priorProgram.append(df_test_false['Description'].values[i-1])
    subsequentProgram.append(df_test_false['Description'].values[i+1])
df_test_false =df_test_false.drop(df_test_false.index[[0]])
df_test_false =df_test_false.drop(df_test_false.index[[len(df_test_false['Description'].values)-1]])
df_test_false['priorProgram'] = priorProgram 
df_test_false['subsequentProgram'] =  subsequentProgram

Description_classified_prior = df_test_false['priorProgram'].values
for i in range(len(df_test_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_test_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_test_false['subsequentProgram'].values
for i in range(len(df_test_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_test_false['subsequentProgram'].values[i]])[0]
df_test_false['priorProgram'] = Description_classified_prior
df_test_false['subsequentProgram'] = Description_classifiedsubsequent
df_test_false.priorProgram= df_test_false.priorProgram.astype(str) 
df_test_false.subsequentProgram= df_test_false.subsequentProgram.astype(str)

#Deleting and string conversion
df_test_false.Date = df_test_false.Date.astype(str)                    #Changing the Date as String 
df_test_false.ProgrammeTheme= df_test_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_test_false.ProgrammeGenre= df_test_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_test_false.StartTime= df_test_false.StartTime.astype(str) 
df_test_false.Length= df_test_false.Length.astype(str)
Y_test_1 = np.array(df_test_false.Impressions)                       #Creating a array for impression Prediction 
del df_test_false['Description']



#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_false['Length'].values]
df_test_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_test_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_test_false['Date'].values]
df_test_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_test_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_test_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_test_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_test_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_test_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_test_false['StartTime'].values
for i in range(len(df_test_false['StartTime'].values)):
    if len(df_test_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_false['StartTime'].values[i].split()[1]
df_test_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_test_false['hour'] = [map(int,i.split(':'))[0] for i in df_test_false['StartTime'].values]#--------------------hour------------------------------->
df_test_false.hour= df_test_false.hour.astype(str)
df_test_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_test_false['StartTime'].values]

Encoding Phase 1

In [123]:
#Decision channel
df_te = df_test_false[:]
Y_test_11 = np.array(df_te.Impressions)  
Y_test_12 = np.array(df_te.Impressions1) #Creating a array for impression Prediction 
del df_te['Impressions']
del df_te['Impressions1']
#del df_te['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_te['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_te['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_te['Length']           #---------------------------------------------------------------------------------------------del length -------------->
#del df_te['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_te['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_te['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_te['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_te['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_te['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_te['Month'] #-------------------------------------------------------------------------------------------------Month------>
#df_te.StartTime= df_te.StartTime.astype(str)
df_te.day= df_te.day.astype(str)
df_te.Month= df_te.Month.astype(str)
Dates = df_te['D'].values
del df_te['D']

In [124]:
#Decision channel
df_tr = df_train_false[:]
Y_train_11 = np.array(df_tr.Impressions) 
Y_train_12 = np.array(df_tr.Impressions1) 
#Creating a array for impression Prediction 
del df_tr['Impressions']
del df_tr['Impressions1']

#del df_tr['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_tr['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_tr['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_tr['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_tr['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_tr['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
#del df_tr['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
#del df_tr['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
del df_tr['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_tr['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_tr['Month'] #-------------------------------------------------------------------------------------------------Month------>
df_tr.day= df_tr.day.astype(str)
df_tr.Month= df_tr.Month.astype(str)

In [125]:
#ENCODING
X_train = df_tr.to_dict('records')
X_test = df_te.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te

In [126]:
#One Hot Encoding 
enc = DictVectorizer()
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_encoded_test =X_encoded_total[len(X_tr):]

In [127]:
#Extra Trees
reg1 = RandomForestRegressor()
reg2  = RandomForestRegressor()
reg1.fit(X_encoded_train,Y_train_11)
reg2.fit(X_encoded_train,Y_train_12)
Y_pred_11 = reg1.predict(X_encoded_test)
Y_pred_12 = reg2.predict(X_encoded_test)

In [103]:
MAPE = []
ME = []
X = []
Y = []
for i in range(len(Y_pred_12)):
    ME.append(Y_pred_12[i] - Y_test_12[i])
    X.append(abs(Y_pred_12[i] - Y_test_12[i]))
    ME.append( Y_pred_12[i] - Y_test_12[i])
    if (Y_pred_12[i] - Y_test_12[i]) == 0:
        MAPE.append(0)
    elif Y_test_12[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_12[i] - Y_test_12[i])/Y_test_12[i]))

In [165]:
df = df_te[:]
df['Actual'] = Y_test_11
df['Predicted'] = Y_pred_11
df['hour'] =df_test_false['hour'].values


C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [150]:
df = df[df['hour'].isin(['1','2','3','4','5','0'])]

In [154]:
df = df[df['hour'].isin(['7','8','9','10','11'])]

In [158]:
df = df[df['hour'].isin(['12','13','14','15','16','17'])]

In [162]:
df = df[df['hour'].isin(['18','19','20'])]

In [166]:
df = df[df['hour'].isin(['21','22','23'])]

In [167]:
Y_test = df['Actual'].values
Y_pred = df['Predicted'].values
MAPE = []
ME = []
for i in range(len(Y_test)):
    ME.append(float(Y_pred[i])- Y_test[i])
    if (Y_pred[i] - Y_test[i]) == 0:
        MAPE.append(0)
    elif Y_test[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred[i] - Y_test[i])/Y_test[i]))

In [168]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print "MAE = ", mean_absolute_error(Y_test,Y_pred)
print "RMSE = ",(mean_squared_error(Y_test,Y_pred))**0.5
print "r2 =", r2_score(Y_test,Y_pred)
print "MAPE",sum(MAPE)/len(Y_pred)
print "ME = ",sum(ME)/len(Y_pred)

MAE =  12.5831269841
RMSE =  17.1030834874
r2 = -0.670970201436
MAPE 4.56975897434
ME =  6.60347619048


# Phase 2 program aggregate to program segment 

## Training model


In [105]:
df_train_P2 = df_train[:]
#df_train_false.Level= df_train_false.Level.astype(str)
#Dropping true Flag values
df_train_P2 = df_train_P2[df_train_P2.EvType != 'Ad Break']
df_train_P2 = df_train_P2[df_train_P2.EvType != 'Promo']
df_train_P2 = df_train_P2[df_train_P2.ProgrammeTheme != 'Teleshopping']


df_train_P2.StartTime= df_train_P2.StartTime.astype(str)
df_train_P2.EndTime= df_train_P2.EndTime.astype(str)
df_train_P2.Level= df_train_P2.Level.astype(str)
#StartTime Dealing
Starttime = df_train_P2['StartTime'].values
for i in range(len(df_train_P2['StartTime'].values)):
    if len(df_train_P2['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_P2['StartTime'].values[i].split()[1]
df_train_P2['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
EndTime = df_train_P2['EndTime'].values
for i in range(len(df_train_P2['EndTime'].values)):
    if len(df_train_P2['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_P2['EndTime'].values[i].split()[1]
df_train_P2['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time

df_train_P2['ST'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_train_P2['StartTime'].values]

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df_train_P2['Level'].values))):
    if df_train_P2['Level'].values[i] == 'True':
        for j in range(i,-1,-1):
            if df_train_P2['Level'].values[j] == 'False':
                priorProgram.append(df_train_P2['Impressions'].values[j])
                priorstarttime.append(df_train_P2['StartTime'].values[j])
                priorendtime.append(df_train_P2['EndTime'].values[j])
                break
                
for i in range(int(len(df_train_P2['Level'].values))):
    if df_train_P2['Level'].values[i] == 'True':
        for j in range(i,int(len(df_train_P2['EvType'].values))):
            if df_train_P2['Level'].values[j] == 'False':
                subsequentProgram.append(df_train_P2['Impressions'].values[j])
                subsequentstarttime.append(df_train_P2['StartTime'].values[j])
                subsequentendtime.append(df_train_P2['EndTime'].values[j])
                break

                
df_train_P2 = df_train_P2[df_train_P2.Level != 'False']
df_train_P2['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_P2['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_P2['priorstarttime'] = priorstarttime
df_train_P2['priorendtime'] =  priorendtime
df_train_P2['subsequentstarttime'] =  subsequentstarttime
df_train_P2['subsequentendtime'] =  subsequentendtime
df_train_P2  = df_train_P2[df_train_P2.priorProgram.notnull()]
df_train_P2  = df_train_P2[df_train_P2.subsequentProgram.notnull()]


df_train_P2.Length= df_train_P2.Length.astype(str)
df_train_P2.StartTime= df_train_P2.StartTime.astype(str)
df_train_P2.priorstarttime= df_train_P2.priorstarttime.astype(str)
df_train_P2.priorendtime= df_train_P2.priorendtime.astype(str)
df_train_P2.subsequentstarttime= df_train_P2.subsequentstarttime.astype(str)
df_train_P2.subsequentendtime= df_train_P2.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['Length'].values]
df_train_P2['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['StartTime'].values]
df_train_P2['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['EndTime'].values]
df_train_P2['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['priorstarttime'].values]
df_train_P2['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['priorendtime'].values]
df_train_P2['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['subsequentstarttime'].values]
df_train_P2['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['subsequentendtime'].values]
df_train_P2['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_train_P2['subsequentProgram'].values)):
    L_1.append(abs(df_train_P2['StartTime'].values[i] - df_train_P2['priorstarttime'].values[i]))
    L_2.append(abs(df_train_P2['priorendtime'].values[i] - df_train_P2['EndTime'].values[i]))
df_train_P2['L_1'] = L_1
df_train_P2['L_2'] = L_2

X_train_2 = []
Y_train_21 = []
Y_train_22 = []
for i in range(len(df_train_P2['priorProgram'].values)):
    X_train_2.append([df_train_P2['priorProgram'].values[i],df_train_P2['ST'].values[i],df_train_P2['subsequentProgram'].values[i],df_train_P2['L_1'].values[i],df_train_P2['L_2'].values[i]])
    Y_train_21.append(df_train_P2['Impressions'].values[i])
    Y_train_22.append(df_train_P2['Impressions1'].values[i])

## Testing model

In [106]:
df_test_P2 = df_test[:]
#df_test_false.Level= df_test_false.Level.astype(str)
#Dropping true Flag values
df_test_P2 = df_test_P2[df_test_P2.EvType != 'Ad Break']
df_test_P2 = df_test_P2[df_test_P2.EvType != 'Promo']
df_test_P2 = df_test_P2[df_test_P2.ProgrammeTheme != 'Teleshopping']


df_test_P2.StartTime= df_test_P2.StartTime.astype(str)
df_test_P2.EndTime= df_test_P2.EndTime.astype(str)
df_test_P2.Level= df_test_P2.Level.astype(str)
#StartTime Dealing
Starttime = df_test_P2['StartTime'].values
for i in range(len(df_test_P2['StartTime'].values)):
    if len(df_test_P2['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_P2['StartTime'].values[i].split()[1]
df_test_P2['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
EndTime = df_test_P2['EndTime'].values
for i in range(len(df_test_P2['EndTime'].values)):
    if len(df_test_P2['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_P2['EndTime'].values[i].split()[1]
df_test_P2['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time
df_test_P2['ST'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_test_P2['StartTime'].values]


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df_test_P2['Level'].values))):
    if df_test_P2['Level'].values[i] == 'True':
        for j in range(i,-1,-1):
            if df_test_P2['Level'].values[j] == 'False':
                priorProgram.append(df_test_P2['Impressions'].values[j])
                priorstarttime.append(df_test_P2['StartTime'].values[j])
                priorendtime.append(df_test_P2['EndTime'].values[j])
                break
                
for i in range(int(len(df_test_P2['Level'].values))):
    if df_test_P2['Level'].values[i] == 'True':
        for j in range(i,int(len(df_test_P2['EvType'].values))):
            if df_test_P2['Level'].values[j] == 'False':
                subsequentProgram.append(df_test_P2['Impressions'].values[j])
                subsequentstarttime.append(df_test_P2['StartTime'].values[j])
                subsequentendtime.append(df_test_P2['EndTime'].values[j])
                break

                
df_test_P2 = df_test_P2[df_test_P2.Level != 'False']
df_test_P2['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_P2['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_P2['priorstarttime'] = priorstarttime
df_test_P2['priorendtime'] =  priorendtime
df_test_P2['subsequentstarttime'] =  subsequentstarttime
df_test_P2['subsequentendtime'] =  subsequentendtime
df_test_P2  = df_test_P2[df_test_P2.priorProgram.notnull()]
df_test_P2  = df_test_P2[df_test_P2.subsequentProgram.notnull()]


df_test_P2.Length= df_test_P2.Length.astype(str)
df_test_P2.StartTime= df_test_P2.StartTime.astype(str)
df_test_P2.priorstarttime= df_test_P2.priorstarttime.astype(str)
df_test_P2.priorendtime= df_test_P2.priorendtime.astype(str)
df_test_P2.subsequentstarttime= df_test_P2.subsequentstarttime.astype(str)
df_test_P2.subsequentendtime= df_test_P2.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['Length'].values]
df_test_P2['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['StartTime'].values]
df_test_P2['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['EndTime'].values]
df_test_P2['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['priorstarttime'].values]
df_test_P2['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['priorendtime'].values]
df_test_P2['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['subsequentstarttime'].values]
df_test_P2['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['subsequentendtime'].values]
df_test_P2['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_test_P2['subsequentProgram'].values)):
    L_1.append(abs(df_test_P2['StartTime'].values[i] - df_test_P2['priorstarttime'].values[i]))
    L_2.append(abs(df_test_P2['priorendtime'].values[i] - df_test_P2['EndTime'].values[i]))
df_test_P2['L_1'] = L_1
df_test_P2['L_2'] = L_2

X_test_2 = []
Y_test_21 = []
Y_test_22 = []
for i in range(len(df_test_P2['priorProgram'].values)):
    X_test_2.append([df_test_P2['priorProgram'].values[i],df_test_P2['ST'].values[i],df_test_P2['subsequentProgram'].values[i],df_test_P2['L_1'].values[i],df_test_P2['L_2'].values[i]])
    Y_test_21.append(df_test_P2['Impressions'].values[i])
    Y_test_22.append(df_test_P2['Impressions1'].values[i])

In [110]:
from sklearn.linear_model import LinearRegression
reg21 = LinearRegression()
reg22 = LinearRegression()
reg21.fit(X_train_2,Y_train_21)
reg22.fit(X_train_2,Y_train_22)
Y_pred_21 = reg21.predict(X_test_2)
Y_pred_22 = reg22.predict(X_test_2)

In [ ]:
len(Y_pred_21)

In [ ]:
df_test_P2['Actual_4+'] = Y_test_21
df_test_P2['predicted_4+'] = Y_pred_21
df_test_P2['Actual_HSM'] = Y_test_22
df_test_P2['predicted_HSM'] = Y_pred_22
df_test_P2['Date'] = df_test_P2['Date']
df = df_test_P2[[ 'Date','Actual_4+','predicted_4+','Actual_HSM','predicted_HSM']]
from pandas import ExcelWriter
writer = ExcelWriter('MTV_P2.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [111]:
MAPE = []
for i in range(len(Y_test_22)):
    ME.append(Y_pred_22[i] - Y_test_22[i])
    if (Y_pred_22[i] - Y_test_22[i]) == 0:
        MAPE.append(0)
    elif Y_test_22[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_22[i] - Y_test_22[i])/Y_test_22[i]))

In [112]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_22,Y_pred_22)
print (mean_squared_error(Y_test_22,Y_pred_22))**0.5
print r2_score(Y_test_22,Y_pred_22)
print sum(MAPE)/len(Y_test_22)
print sum(ME)/len(Y_test_22)

20.816149853
28.3911702836
0.245453492156
4.13574803919
12.3727710841


# Phase3, Program segment to promo level

## Training the data 

In [113]:
df_train_P3 = df_train[:]
df_train_P3 = df_train_P3[df_train_P3.Level != False]
df_train_P3 =  df_train_P3[df_train_P3.EvType != 'Ad Break']
df_train_P3 =  df_train_P3[df_train_P3.ProgrammeTheme != 'Teleshopping']

In [114]:

################################################################
df_train_P3.StartTime= df_train_P3.StartTime.astype(str)
df_train_P3.EndTime= df_train_P3.EndTime.astype(str)
#StartTime Dealing
Starttime = df_train_P3['StartTime'].values
for i in range(len(df_train_P3['StartTime'].values)):
    if len(df_train_P3['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_P3['StartTime'].values[i].split()[1]
df_train_P3['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_train_P3['EndTime'].values
for i in range(len(df_train_P3['EndTime'].values)):
    if len(df_train_P3['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_P3['EndTime'].values[i].split()[1]
df_train_P3['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_train_P3['EvType'].values))):
    if df_train_P3['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_train_P3['EvType'].values[j] == 'Program':
                priorProgram.append(df_train_P3['Impressions'].values[j])
                priorstarttime.append(df_train_P3['StartTime'].values[j])
                priorendtime.append(df_train_P3['EndTime'].values[j])
                break
for i in range(int(len(df_train_P3['EvType'].values))):
    if df_train_P3['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_train_P3['EvType'].values))):
            if df_train_P3['EvType'].values[j] == 'Program':
                subsequentProgram.append(df_train_P3['Impressions'].values[j])
                subsequentstarttime.append(df_train_P3['StartTime'].values[j])
                subsequentendtime.append(df_train_P3['EndTime'].values[j])
                break

                
df_train_P3 = df_train_P3[df_train_P3.EvType != 'Program']
df_train_P3['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_P3['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_P3['priorstarttime'] = priorstarttime
df_train_P3['priorendtime'] =  priorendtime
df_train_P3['subsequentstarttime'] =  subsequentstarttime
df_train_P3['subsequentendtime'] =  subsequentendtime
df_train_P3  = df_train_P3[df_train_P3.priorProgram.notnull()]
df_train_P3  = df_train_P3[df_train_P3.subsequentProgram.notnull()]  


df_train_P3.Length= df_train_P3.Length.astype(str)
df_train_P3.StartTime= df_train_P3.StartTime.astype(str)
df_train_P3.priorstarttime= df_train_P3.priorstarttime.astype(str)
df_train_P3.priorendtime= df_train_P3.priorendtime.astype(str)
df_train_P3.subsequentstarttime= df_train_P3.subsequentstarttime.astype(str)
df_train_P3.subsequentendtime= df_train_P3.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['Length'].values]
df_train_P3['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['StartTime'].values]
df_train_P3['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['EndTime'].values]
df_train_P3['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['priorstarttime'].values]
df_train_P3['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['priorendtime'].values]
df_train_P3['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['subsequentstarttime'].values]
df_train_P3['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['subsequentendtime'].values]
df_train_P3['subsequentendtime'] = subsequentendtime_seconds




L_1 = []
L_2 = []
for i in range(len(df_train_P3['subsequentProgram'].values)):
    L_1.append(df_train_P3['StartTime'].values[i] - df_train_P3['priorendtime'].values[i])
    L_2.append(df_train_P3['subsequentstarttime'].values[i] - df_train_P3['EndTime'].values[i])
df_train_P3['L_1'] = L_1
df_train_P3['L_2'] = L_2



X_train_3 = []
Y_train_31 = []
Y_train_32 = []
for i in range(len(priorProgram)):
    X_train_3.append([df_train_P3['priorProgram'].values[i],df_train_P3['Length'].values[i],df_train_P3['subsequentProgram'].values[i],df_train_P3['L_1'].values[i],df_train_P3['L_2'].values[i]])
    Y_train_31.append(df_train_P3['Impressions'].values[i])
    Y_train_32.append(df_train_P3['Impressions1'].values[i])
    

## Testing

In [115]:
df_test_P3 = df_test[:]
df_test_P3 = df_test_P3[df_test_P3.Level != False]
df_test_P3 =  df_test_P3[df_test_P3.EvType != 'Ad Break']
df_test_P3 =  df_test_P3[df_test_P3.ProgrammeTheme != 'Teleshopping']

In [116]:

################################################################
df_test_P3.StartTime= df_test_P3.StartTime.astype(str)
df_test_P3.EndTime= df_test_P3.EndTime.astype(str)
#StartTime Dealing
Starttime = df_test_P3['StartTime'].values
for i in range(len(df_test_P3['StartTime'].values)):
    if len(df_test_P3['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_P3['StartTime'].values[i].split()[1]
df_test_P3['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_test_P3['EndTime'].values
for i in range(len(df_test_P3['EndTime'].values)):
    if len(df_test_P3['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_P3['EndTime'].values[i].split()[1]
df_test_P3['EndTime'] = EndTime #-----------------------------------------------------------------------------------------------End time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_test_P3['EvType'].values))):
    if df_test_P3['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_test_P3['EvType'].values[j] == 'Program':
                priorProgram.append(df_test_P3['Impressions'].values[j])
                priorstarttime.append(df_test_P3['StartTime'].values[j])
                priorendtime.append(df_test_P3['EndTime'].values[j])
                break
for i in range(int(len(df_test_P3['EvType'].values))):
    if df_test_P3['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_test_P3['EvType'].values))):
            if df_test_P3['EvType'].values[j] == 'Program':
                subsequentProgram.append(df_test_P3['Impressions'].values[j])
                subsequentstarttime.append(df_test_P3['StartTime'].values[j])
                subsequentendtime.append(df_test_P3['EndTime'].values[j])
                break

                
df_test_P3 = df_test_P3[df_test_P3.EvType != 'Program']
df_test_P3['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_P3['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_P3['priorstarttime'] = priorstarttime
df_test_P3['priorendtime'] =  priorendtime
df_test_P3['subsequentstarttime'] =  subsequentstarttime
df_test_P3['subsequentendtime'] =  subsequentendtime
df_test_P3  = df_test_P3[df_test_P3.priorProgram.notnull()]
df_test_P3  = df_test_P3[df_test_P3.subsequentProgram.notnull()]  


df_test_P3.Length= df_test_P3.Length.astype(str)
df_test_P3.StartTime= df_test_P3.StartTime.astype(str)
df_test_P3.priorstarttime= df_test_P3.priorstarttime.astype(str)
df_test_P3.priorendtime= df_test_P3.priorendtime.astype(str)
df_test_P3.subsequentstarttime= df_test_P3.subsequentstarttime.astype(str)
df_test_P3.subsequentendtime= df_test_P3.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['Length'].values]
df_test_P3['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['StartTime'].values]
df_test_P3['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['EndTime'].values]
df_test_P3['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['priorstarttime'].values]
df_test_P3['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['priorendtime'].values]
df_test_P3['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['subsequentstarttime'].values]
df_test_P3['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['subsequentendtime'].values]
df_test_P3['subsequentendtime'] = subsequentendtime_seconds




L_1 = []
L_2 = []
for i in range(len(df_test_P3['subsequentProgram'].values)):
    L_1.append(df_test_P3['StartTime'].values[i] - df_test_P3['priorendtime'].values[i])
    L_2.append(df_test_P3['subsequentstarttime'].values[i] - df_test_P3['EndTime'].values[i])
df_test_P3['L_1'] = L_1
df_test_P3['L_2'] = L_2



X_test_3 = []
Y_test_31 = []
Y_test_32 = []
for i in range(len(priorProgram)):
    X_test_3.append([df_test_P3['priorProgram'].values[i],df_test_P3['Length'].values[i],df_test_P3['subsequentProgram'].values[i],df_test_P3['L_1'].values[i],df_test_P3['L_2'].values[i]])
    Y_test_31.append(df_test_P3['Impressions'].values[i])
    Y_test_32.append(df_test_P3['Impressions1'].values[i])
    

In [117]:
from sklearn.linear_model import LinearRegression
reg31 = LinearRegression()
reg32 = LinearRegression()
reg31.fit(X_train_3,Y_train_31)
reg32.fit(X_train_3,Y_train_32)
Y_pred_31 =  reg31.predict(X_test_3)
Y_pred_32 =  reg32.predict(X_test_3)

In [ ]:
df_test_P3['Actual_4+'] = Y_test_31
df_test_P3['predicted_4+'] = Y_pred_31
df_test_P3['Actual_HSM'] = Y_test_32
df_test_P3['predicted_HSM'] = Y_pred_32
df_test_P3['Date'] = df_test_P3['Date']
df = df_test_P3[[ 'Date','Actual_4+','predicted_4+','Actual_HSM','predicted_HSM']]
from pandas import ExcelWriter
writer = ExcelWriter('MTV_P3.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [118]:
MAPE = []
ME = []
X = []
Y = []
for i in range(len(Y_test_32)):
    ME.append(Y_pred_32[i] - Y_test_32[i])
    if (Y_pred_32[i] - Y_test_32[i]) == 0:
        MAPE.append(0)
    elif Y_test_32[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_32[i] - Y_test_32[i])/Y_test_32[i]))
        

In [119]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_32,Y_pred_32)
print r2_score(Y_test_32,Y_pred_32)
print (mean_squared_error(Y_test_32,Y_pred_32))**0.5
print sum(MAPE)/len(Y_test_32)
print sum(ME)/len(Y_test_32)

20.8134500188
0.118281379944
28.4925268245
1.41431075011
3.22771205692


# bypass Phase Analysis 

In [ ]:
df_train_bp = df_train[:]
df_train_bp =  df_train_bp[df_train_bp.EvType != 'Ad Break']
df_train_bp =  df_train_bp[df_train_bp.ProgrammeTheme != 'Teleshopping']
################################################################
df_train_bp.StartTime= df_train_bp.StartTime.astype(str)
df_train_bp.EndTime=   df_train_bp.EndTime.astype(str)
#StartTime Dealing
Starttime = df_train_bp['StartTime'].values
for i in range(len(df_train_bp['StartTime'].values)):
    if len(df_train_bp['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_bp['StartTime'].values[i].split()[1]
df_train_bp['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_train_bp['EndTime'].values
for i in range(len(df_train_bp['EndTime'].values)):
    if len(df_train_bp['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_bp['EndTime'].values[i].split()[1]
df_train_bp['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_train_bp['EvType'].values))):
    if df_train_bp['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_train_bp['Level'].values[j] == False:
                priorProgram.append(df_train_bp['Impressions'].values[j])
                priorstarttime.append(df_train_bp['StartTime'].values[j])
                priorendtime.append(df_train_bp['EndTime'].values[j])
                break
for i in range(int(len(df_train_bp['EvType'].values))):
    if df_train_bp['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_train_bp['EvType'].values))):
            if df_train_bp['Level'].values[j] == False:
                subsequentProgram.append(df_train_bp['Impressions'].values[j])
                subsequentstarttime.append(df_train_bp['StartTime'].values[j])
                subsequentendtime.append(df_train_bp['EndTime'].values[j])
                break

                
df_train_bp =  df_train_bp[df_train_bp.EvType != 'Program']
df_train_bp['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_bp['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_bp['priorstarttime'] = priorstarttime
df_train_bp['priorendtime'] =  priorendtime
df_train_bp['subsequentstarttime'] =  subsequentstarttime
df_train_bp['subsequentendtime'] =  subsequentendtime
df_train_bp  = df_train_bp[df_train_bp.priorProgram.notnull()]
df_train_bp  = df_train_bp[df_train_bp.subsequentProgram.notnull()]  


df_train_bp.Length= df_train_bp.Length.astype(str)
df_train_bp.StartTime= df_train_bp.StartTime.astype(str)
df_train_bp.priorstarttime= df_train_bp.priorstarttime.astype(str)
df_train_bp.priorendtime= df_train_bp.priorendtime.astype(str)
df_train_bp.subsequentstarttime= df_train_bp.subsequentstarttime.astype(str)
df_train_bp.subsequentendtime= df_train_bp.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['Length'].values]
df_train_bp['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['StartTime'].values]
df_train_bp['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['EndTime'].values]
df_train_bp['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['priorstarttime'].values]
df_train_bp['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['priorendtime'].values]
df_train_bp['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['subsequentstarttime'].values]
df_train_bp['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['subsequentendtime'].values]
df_train_bp['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_train_bp['subsequentProgram'].values)):
    L_1.append(abs(df_train_bp['StartTime'].values[i] - df_train_bp['priorstarttime'].values[i]))
    L_2.append(abs(df_train_bp['priorendtime'].values[i] - df_train_bp['EndTime'].values[i]))
df_train_bp['L_1'] = L_1
df_train_bp['L_2'] = L_2

X_train_bp = []
Y_train_bp = []
for i in range(len(priorProgram)):
    X_train_bp.append([df_train_bp['priorProgram'].values[i],df_train_bp['subsequentProgram'].values[i],df_train_bp['L_1'].values[i],df_train_bp['L_2'].values[i]])
    Y_train_bp.append(df_train_bp['Impressions'].values[i])


## TESTING

In [ ]:
df_test_bp = df_test[:]
df_test_bp =  df_test_bp[df_test_bp.EvType != 'Ad Break']
df_test_bp =  df_test_bp[df_test_bp.ProgrammeTheme != 'Teleshopping']
################################################################
df_test_bp.StartTime= df_test_bp.StartTime.astype(str)
df_test_bp.EndTime=   df_test_bp.EndTime.astype(str)
#StartTime Dealing
Starttime = df_test_bp['StartTime'].values
for i in range(len(df_test_bp['StartTime'].values)):
    if len(df_test_bp['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_bp['StartTime'].values[i].split()[1]
df_test_bp['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_test_bp['EndTime'].values
for i in range(len(df_test_bp['EndTime'].values)):
    if len(df_test_bp['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_bp['EndTime'].values[i].split()[1]
df_test_bp['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_test_bp['EvType'].values))):
    if df_test_bp['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_test_bp['Level'].values[j] == False:
                priorProgram.append(df_test_bp['Impressions'].values[j])
                priorstarttime.append(df_test_bp['StartTime'].values[j])
                priorendtime.append(df_test_bp['EndTime'].values[j])
                break
for i in range(int(len(df_test_bp['EvType'].values))):
    if df_test_bp['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_test_bp['EvType'].values))):
            if df_test_bp['Level'].values[j] == False:
                subsequentProgram.append(df_test_bp['Impressions'].values[j])
                subsequentstarttime.append(df_test_bp['StartTime'].values[j])
                subsequentendtime.append(df_test_bp['EndTime'].values[j])
                break

                
df_test_bp =  df_test_bp[df_test_bp.EvType != 'Program']
df_test_bp['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_bp['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_bp['priorstarttime'] = priorstarttime
df_test_bp['priorendtime'] =  priorendtime
df_test_bp['subsequentstarttime'] =  subsequentstarttime
df_test_bp['subsequentendtime'] =  subsequentendtime
df_test_bp  = df_test_bp[df_test_bp.priorProgram.notnull()]
df_test_bp  = df_test_bp[df_test_bp.subsequentProgram.notnull()]  


df_test_bp.Length= df_test_bp.Length.astype(str)
df_test_bp.StartTime= df_test_bp.StartTime.astype(str)
df_test_bp.priorstarttime= df_test_bp.priorstarttime.astype(str)
df_test_bp.priorendtime= df_test_bp.priorendtime.astype(str)
df_test_bp.subsequentstarttime= df_test_bp.subsequentstarttime.astype(str)
df_test_bp.subsequentendtime= df_test_bp.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['Length'].values]
df_test_bp['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['StartTime'].values]
df_test_bp['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['EndTime'].values]
df_test_bp['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['priorstarttime'].values]
df_test_bp['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['priorendtime'].values]
df_test_bp['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['subsequentstarttime'].values]
df_test_bp['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['subsequentendtime'].values]
df_test_bp['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_test_bp['subsequentProgram'].values)):
    L_1.append(abs(df_test_bp['StartTime'].values[i] - df_test_bp['priorstarttime'].values[i]))
    L_2.append(abs(df_test_bp['priorendtime'].values[i] - df_test_bp['EndTime'].values[i]))
df_test_bp['L_1'] = L_1
df_test_bp['L_2'] = L_2

X_test_bp = []
Y_test_bp = []
for i in range(len(priorProgram)):
    X_test_bp.append([df_test_bp['priorProgram'].values[i],df_test_bp['subsequentProgram'].values[i],df_test_bp['L_1'].values[i],df_test_bp['L_2'].values[i]])
    Y_test_bp.append(df_test_bp['Impressions'].values[i])


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train_bp,Y_train_bp)
Y_pred_bp =  reg.predict(X_test_bp)

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y_test_bp,Y_pred_bp)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_bp,Y_pred_bp)
print mean_squared_error(Y_test_bp,Y_pred_bp)
print r2_score(Y_test_bp,Y_pred_bp)
print (mean_squared_error(Y_test_bp,Y_pred_bp))**0.5

In [ ]:
Y_test_bp = np.array(Y_test_bp)
1-sum(np.square(Y_test_bp-Y_pred_bp))/sum(np.square(Y_test_bp-Y_test_bp.mean()))

# All Phase Analysis

In [ ]:
# All phase analysis
df = df_test
df['Numbers'] = np.arange(len(df))
df = df[df.ProgrammeTheme != 'Teleshopping']
df = df[df.EvType != 'Ad Break']
#del df['Impressions']

In [ ]:
df.StartTime= df.StartTime.astype(str)
df.EndTime=   df.EndTime.astype(str)
#StartTime Dealing
Starttime = df['StartTime'].values
for i in range(len(df['StartTime'].values)):
    if len(df['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df['StartTime'].values[i].split()[1]
df['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df['EndTime'].values
for i in range(len(df['EndTime'].values)):
    if len(df['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df['EndTime'].values[i].split()[1]
df['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


In [ ]:
#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorProgramnumber = []
subsequentProgramnumber = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df['EvType'].values))):
    if df['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df['Level'].values[j] == False:
                priorProgram.append(df['Description'].values[j])
                priorProgramnumber.append(df['Numbers'].values[j])
                priorstarttime.append(df['StartTime'].values[j])
                priorendtime.append(df['EndTime'].values[j])
                break
for i in range(int(len(df['EvType'].values))):
    if df['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df['EvType'].values))):
            if df['Level'].values[j] == False:
                subsequentProgram.append(df['Description'].values[j])
                subsequentProgramnumber.append(df['Numbers'].values[j])
                subsequentstarttime.append(df['StartTime'].values[j])
                subsequentendtime.append(df['EndTime'].values[j])
                break

In [ ]:
df =  df[df.EvType != 'Program']
df['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df['priorstarttime'] = priorstarttime
df['priorendtime'] =  priorendtime
df['subsequentstarttime'] =  subsequentstarttime
df['subsequentendtime'] =  subsequentendtime
df['subsequentProgramnumber'] = subsequentProgramnumber
df['priorProgramnumber'] = priorProgramnumber
df  = df[df.priorProgram.notnull()]
df  = df[df.subsequentProgram.notnull()]  

In [ ]:

df.Length= df.Length.astype(str)
df.StartTime= df.StartTime.astype(str)
df.priorstarttime= df.priorstarttime.astype(str)
df.priorendtime= df.priorendtime.astype(str)
df.subsequentstarttime= df.subsequentstarttime.astype(str)
df.subsequentendtime= df.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['Length'].values]
df['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['StartTime'].values]
df['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['EndTime'].values]
df['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['priorstarttime'].values]
df['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['priorendtime'].values]
df['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['subsequentstarttime'].values]
df['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['subsequentendtime'].values]
df['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df['subsequentProgram'].values)):
    L_1.append(abs(df['StartTime'].values[i] - df['priorstarttime'].values[i]))
    L_2.append(abs(df['priorendtime'].values[i] - df['EndTime'].values[i]))
df['L_1'] = L_1
df['L_2'] = L_2

In [ ]:
df_test_false = df_test[:]
#df_test_false.Level= df_test_false.Level.astype(str)
#Dropping true Flag values
df_test_false['Numbers'] = np.arange(len(df_test_false))
df_test_false = df_test_false[df_test_false.Level != True]
df_test_false = df_test_false[df_test_false.ProgrammeTheme != 'Teleshopping']
#deleting unwanted columns
del df_test_false['BreakCode']                                  
del df_test_false['EvType']
del df_test_false['PromoCategory']
del df_test_false['PromoType']
del df_test_false['EndTime']
del df_test_false['Level'] 
#Taggin 'Movies / Plays' into Movie
Description = df_test_false['Description'].values
for i in range(len(df_test_false['Description'].values)):
    if df_test_false['ProgrammeTheme'].values[i] == 'Movies / Plays':
         Description[i] = 'Movies'
    if '1 Hour Special' in df_test_false['Description'].values[i]:
        Description[i] = df_test_false['Description'].values[i].replace('- 1 Hour Special','').strip()       
df_test_false['Description'] = Description

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_test_false['Description'].values)-1):
    priorProgram.append(df_test_false['Description'].values[i-1])
    subsequentProgram.append(df_test_false['Description'].values[i+1])
df_test_false =df_test_false.drop(df_test_false.index[[0]])
df_test_false =df_test_false.drop(df_test_false.index[[len(df_test_false['Description'].values)-1]])
df_test_false['priorProgram'] = priorProgram 
df_test_false['subsequentProgram'] =  subsequentProgram

Description_classified_prior = df_test_false['priorProgram'].values
for i in range(len(df_test_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_test_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_test_false['subsequentProgram'].values
for i in range(len(df_test_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_test_false['subsequentProgram'].values[i]])[0]
df_test_false['priorProgram'] = Description_classified_prior
df_test_false['subsequentProgram'] = Description_classifiedsubsequent
df_test_false.priorProgram= df_test_false.priorProgram.astype(str) 
df_test_false.subsequentProgram= df_test_false.subsequentProgram.astype(str)

#Deleting and string conversion
df_test_false.Date = df_test_false.Date.astype(str)                    #Changing the Date as String 
df_test_false.ProgrammeTheme= df_test_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_test_false.ProgrammeGenre= df_test_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_test_false.StartTime= df_test_false.StartTime.astype(str) 
df_test_false.Length= df_test_false.Length.astype(str)
Y_test_1 = np.array(df_test_false.Impressions)                       #Creating a array for impression Prediction 
del df_test_false['Impressions']
del df_test_false['Description']



#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_false['Length'].values]
df_test_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_test_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_test_false['Date'].values]
df_test_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_test_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_test_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_test_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_test_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_test_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_test_false['StartTime'].values
for i in range(len(df_test_false['StartTime'].values)):
    if len(df_test_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_false['StartTime'].values[i].split()[1]
df_test_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_test_false['hour'] = [map(int,i.split(':'))[0] for i in df_test_false['StartTime'].values]#--------------------hour------------------------------->
df_test_false.hour= df_test_false.hour.astype(str)
df_test_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_test_false['StartTime'].values]


In [ ]:
#Decision channel
df_tr = df_train_false[:]

#del df_tr['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_tr['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_tr['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_tr['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_tr['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_tr['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_tr['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_tr['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_tr['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_tr['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_tr['Month'] #-------------------------------------------------------------------------------------------------Month------>

In [ ]:
#Decision channel
df_te = df_test_false[:]
del df_te['Numbers']
#del df_te['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_te['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
del df_te['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_te['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_te['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_te['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_te['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_te['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_te['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_te['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_te['Month'] #-------------------------------------------------------------------------------------------------Month------>

In [ ]:
#ENCODING
X_train = df_tr.to_dict('records')
X_test = df_te.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te

In [ ]:
#Extra Trees
reg1 = ExtraTreesRegressor()
reg1.fit(X_encoded_train,Y_train_1)
Y_pred1 = reg1.predict(X_encoded_test)
from sklearn.metrics import r2_score
r2_score(Y_pred1 ,Y_test_1)

In [ ]:
df_test_false['predicted'] = Y_pred1
df_false = df_test_false[['predicted']]

In [ ]:
#df = df[df.priorProgramnumber != 0]
#df = df[df.subsequentProgramnumber != 12372]

In [ ]:
priorProgram = []
subsequentProgram = []
for i in df['priorProgramnumber'].values:
    priorProgram.append(df_false.to_dict()['predicted'][i])
    subsequentProgram.append(df_false.to_dict()['predicted'][i])
df['priorProgram'] = priorProgram
df['subsequentProgram'] = subsequentProgram

In [ ]:
X_test_ca = []
for i in range(len(priorProgram)):
    X_test_ca.append([df['priorProgram'].values[i],df['subsequentProgram'].values[i],df['L_1'].values[i],df['L_2'].values[i]])

In [ ]:
from sklearn.linear_model import LinearRegression
reg = RandomForestRegressor()
reg.fit(X_train_bp,Y_train_bp)
Y_pred_ca =  reg.predict(X_test_ca)

In [ ]:
r2_score(Y_pred_ca,df['Impressions'].values)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print 'MAE = ',mean_absolute_error(df['Impressions'].values,Y_pred_ca)
print 'MSE = ',mean_squared_error(df['Impressions'].values,Y_pred_ca)
print 'RMSE = ',(mean_squared_error(df['Impressions'].values,Y_pred_ca))**0.5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA
A = 200
X = np.arange(A)
Y = df['Impressions'].values[:A]
x = np.arange(A)
y = Y_pred_ca[:A]
####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(x, y, 'Red', linewidth=2)
sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
plt.plot(X,Y)
plt.plot(x,y)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA

A = len(Y_test_1)
#X = np.arange(A)
#Y = Y_test_1[:A]
x = np.arange(A)
y = Y_pred1- Y_test_1
####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(x, y, 'Red', linewidth=2)
#sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
#plt.plot(X,Y)
plt.scatter(x,y)

plt.tight_layout()
plt.show()

In [ ]:
len(Y_pred1)